# Choropleth map

In [13]:
import openpyxl
print(openpyxl.__version__)


3.1.5


In [15]:
import pandas as pd
import duckdb

df = pd.read_excel(
    "data/resultat-ansokningsomgang-2024.xlsx", sheet_name="Tabell 3", skiprows=5
)
df.head() 

,Utbildningsområde,SUN5 inriktning,SUN5 inriktning namn,Utbildningsnamn,Beslut,Diarienummer,Flera kommuner,Antal kommuner,Län,Kommun,...,Sökta utbildningsomgångar,Beviljade utbildningsomgångar,Sökta platser per utbildningsomgång,Sökta platser totalt,Beviljade platser utbildningsomgång 1,Beviljade platser utbildningsomgång 2,Beviljade platser utbildningsomgång 3,Beviljade platser utbildningsomgång 4,Beviljade platser utbildningsomgång 5,Beviljade platser totalt
0,Data/IT,481ab,"Utbildningar till programmerare, spel",AI Programmer,Avslag,MYH 2024/3742,Nej,1,Västerbotten,Umeå,...,3,0,30,90,0,0,0,0,0,0
1,Data/IT,481ab,"Utbildningar till programmerare, spel","Forsbergs, Spelutvecklare – Game Programming",Avslag,MYH 2024/4174,Nej,1,Stockholm,Stockholm,...,3,0,20,60,0,0,0,0,0,0
2,Data/IT,481ab,"Utbildningar till programmerare, spel",Futuregames Game Programmer Specialization,Avslag,MYH 2024/3029,Ja,2,Flera kommuner,Flera kommuner,...,3,0,30,90,0,0,0,0,0,0
3,Data/IT,481ab,"Utbildningar till programmerare, spel",Game Programmer,Avslag,MYH 2024/3425,Nej,1,Blekinge,Karlshamn,...,3,0,25,75,0,0,0,0,0,0
4,Data/IT,481ab,"Utbildningar till programmerare, spel",Spelprogrammerare,Beviljad,MYH 2024/3852,Nej,1,Skåne,Malmö,...,4,3,35,140,35,35,35,0,0,105


In [16]:
decisions = df["Beslut"].value_counts()
approved, total = decisions["Beviljad"], decisions.sum()
approved, total

(np.int64(344), np.int64(1272))

In [19]:
df_regions = duckdb.query(
    """
    SELECT 
        län,
        CAST(COUNT_IF(beslut = 'Beviljad') AS INTEGER) AS Beviljade
    FROM df 
    WHERE län != 'Flera kommuner'
    GROUP BY län
    ORDER BY 
        beviljade DESC, län ASC
"""
).df()

df_regions

,Län,Beviljade
0,Stockholm,82
1,Västra Götaland,63
2,Skåne,37
3,Östergötland,16
4,Jönköping,13
5,Örebro,10
6,Västernorrland,6
7,Gävleborg,5
8,Kalmar,5
9,Kronoberg,5


In [20]:
df_regions["Län"].values

array(['Stockholm', 'Västra Götaland', 'Skåne', 'Östergötland',
       'Jönköping', 'Örebro', 'Västernorrland', 'Gävleborg', 'Kalmar',
       'Kronoberg', 'Västmanland', 'Dalarna', 'Halland', 'Norrbotten',
       'Uppsala', 'Värmland', 'Västerbotten', 'Blekinge', 'Södermanland',
       'Jämtland'], dtype=object)

In [21]:
import json

with open("assets/swedish_regions.geojson", "r") as file:
    json_data = json.load(file)


json_data.get("features")[0].get("properties")

{'admin_level': '4',
 'boundary': 'administrative',
 'name': 'Gotlands lÃ¤n',
 'ref:se:lÃ¤n:bokstav': 'I',
 'ref:se:lÃ¤nskod': '09',
 'type': 'multipolygon'}

In [22]:
properties = [feature.get("properties") for feature in json_data.get("features")]
region_codes = {
    property.get("name"): property.get("ref:se:länskod") for property in properties
}

region_codes

{'Gotlands lÃ¤n': None,
 'Kalmar lÃ¤n': None,
 'Blekinge lÃ¤n': None,
 'JÃ¤mtlands lÃ¤n': None,
 'VÃ¤sterbottens lÃ¤n': None,
 'Kronobergs lÃ¤n': None,
 'VÃ¤stra GÃ¶talands lÃ¤n': None,
 'VÃ¤rmlands lÃ¤n': None,
 'Dalarnas lÃ¤n': None,
 'SÃ¶dermanlands lÃ¤n': None,
 'JÃ¶nkÃ¶pings lÃ¤n': None,
 'Ã–stergÃ¶tlands lÃ¤n': None,
 'Stockholms lÃ¤n': None,
 'VÃ¤stmanlands lÃ¤n': None,
 'Uppsala lÃ¤n': None,
 'GÃ¤vleborgs lÃ¤n': None,
 'VÃ¤sternorrlands lÃ¤n': None,
 'Ã–rebro lÃ¤n': None,
 'Norrbottens lÃ¤n': None,
 'Hallands lÃ¤n': None,
 'SkÃ¥ne lÃ¤n': None}

In [23]:
from difflib import get_close_matches

get_close_matches("appel", ["ape", "apple", "peach", "puppy"])
['apple', 'ape']

['apple', 'ape']

In [24]:
get_close_matches("appel", ["ape", "apple", "peach", "puppy"], n = 1)[0]

'apple'

In [25]:
df_regions["Län"].iloc[0]

'Stockholm'

In [26]:
region = get_close_matches(df_regions["Län"].iloc[0], region_codes.keys())[0]
region

'Stockholms lÃ¤n'

In [27]:
region_codes[region]

In [30]:
from difflib import get_close_matches

region_codes_map = []
for region in df_regions["Län"]:
    matches = get_close_matches(region, region_codes.keys(), n=1)
    if matches:  # Kolla om det finns några matchningar
        region_name = matches[0]
        code = region_codes[region_name]
        region_codes_map.append(code)
    else:
        # Om ingen matchning hittades, kan du antingen lägga till ett standardvärde eller hoppa över
        region_codes_map.append(None)  # Eller något annat standardvärde


## Draw map

In [31]:
import numpy as np 

log_approved = np.log(df_regions["Beviljade"] + 1)
log_approved.values

array([4.41884061, 4.15888308, 3.63758616, 2.83321334, 2.63905733,
       2.39789527, 1.94591015, 1.79175947, 1.79175947, 1.79175947,
       1.79175947, 1.60943791, 1.60943791, 1.60943791, 1.60943791,
       1.60943791, 1.60943791, 1.38629436, 1.38629436, 1.09861229])

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(
    go.Choroplethmapbox(
        geojson=json_data,
        locations=region_codes_map,
        z=log_approved,
        featureidkey="properties.ref:se:länskod",
        colorscale="Blues",
        showscale=False,
        customdata=df_regions["Beviljade"],
        text=df_regions["Län"],
        hovertemplate="<b>%{text}</b><br>Beviljade utbildningar %{customdata}<extra></extra>",
        marker_line_width=0.3,
    )
)

fig.update_layout(
    mapbox=dict(style="white-bg", zoom=3.3, center=dict(lat=62.6952, lon=13.9149)),
    width=470,
    height=500,
    margin=dict(r=0, t=50, l=0, b=0),
    title=dict(
        text=f"""
                <b>Antal beviljade</b>
                <br>utbildningar per län
                <br>inom YH i Sverige för 
                <br>omgång 2024. Ju mörkare 
                <br>blå färg, desto fler
                <br>beviljade utbildningar
                <br>
                <br><b>{approved}</b> av totalt <b>{total}</b>
                <br>ansökningar har
                <br>godkänts, vilket innebär 
                <br><b>{approved/total*100:.0f}%</b> beviljandegrad
                <br>
                <br><b>I ledningen är</b>
                <br>1. Stockholm, 
                <br>2. Västra Götaland
                <br>3. Skåne""",
        x=0.06,
        y=0.75,
        font=dict(size=13),
    ),
)


fig.show()

## Exercise for reader

1. filter on different education areas 
2. approved educations on municipality level